In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **Agentic Rag**

**질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트**

Agent 를 활용하여 RAG 를 수행한다면 Agentic RAG 라고 합니다. 

### **웹 검색도구: Tavily Search**

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search  = TavilySearchResults(k=6)      # 검색 결과를 6개까지 가져오겠다

In [3]:
search.invoke("대구에 동성로 중앙떡볶이 전화번호는 무엇인가요?")

[{'url': 'https://blog.naver.com/PostView.nhn?blogId=oo5466&logNo=223658142841&redirect=Dlog&widgetTypeCall=true',
  'content': '| 대구[대구] 동성로 떡볶이 맛집 "중앙떡볶이" + 포장짝누・2024. 11. 13. 2:09URL 복사이웃추가본문 기타 기능공유하기신고하기안녕하세요 짝누입니다~:)\u200b오늘은 대구 떡볶이 맛집으로 유명한 "중앙떡볶이"에 대해 기록을 하려고 합니다!\u200b｡ﾟ•┈୨♡୧┈•ﾟ｡\u200b▪ 📱위치:대구 중구 동성로2길 81 중앙떡볶이\u200b▪ ⏰영업 시간: 오전 11시 30분 ~ 오후 6시▪ 브레이크 타임: 오후 2시 ~ 오후 5시\u200b▪ 📞전화번호:053-424-7692\u200b｡ﾟ•┈୨♡୧┈•ﾟ｡\u200b외관입니다.\u200b제가 방문했을 때 직원분들 계신 곳과 문쪽에 사람들이 서있어 문 옆쪽에 서 계신 분께 "중앙떡볶이 먹으려면 여기에 줄 서면 되나요?"라고 물어보니 아니라고 가게로 바로 들어가면 된다고 말씀하시면서 죄송하다고 하시더라고요😃대구분들 정말 친절하신 것 같아요!!\u200b별일도 아닌 일에도 사과를 하셔서 대구분들은 다들 왜 이렇게 친절하냐며 이야기를 나눴습니다🤭😘\u200b아참 그리고!포장하시는 분들은 문 옆에 있는 오픈된 [...] 안녕하세요 짝누입니다~:)\n\n\u200b\n\n오늘은 대구 떡볶이 맛집으로 유명한 "중앙떡볶이"에 대해 기록을 하려고 합니다!\n\n\u200b\n\n｡ﾟ•┈୨♡୧┈•ﾟ｡\n\n\u200b\n\n▪ 📱위치: 대구 중구 동성로2길 81 중앙떡볶이\n\n\u200b\n\n▪ ⏰영업 시간: 오전 11시 30분 ~ 오후 6시\n\n▪ 브레이크 타임: 오후 2시 ~ 오후 5시\n\n\u200b\n\n▪ 📞전화번호: 053-424-7692\n\n\u200b\n\n｡ﾟ•┈୨♡୧┈•ﾟ｡\n\n\u200b\n\n외관입니다.\n\n\u200b\n\n제가 방문했을 때 직원분들 계

## **Retriever**

PDF 문서를 FAISS DB 에 저장하고 조회하는 retriever 를 생성합니다.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

In [5]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader('data/SPRI_AI_Brief_2023년12월호_F.pdf')
docs = loader.load()

In [6]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

In [7]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [8]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [9]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [10]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke('삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘')

[Document(id='5f6c4825-737b-4335-a182-899739b6ef92', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'page': 12}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정

`create_retriever_tool()` 함수로 `retriever` 를 도구로 변환합니다.

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,              # 리트리버
    name='pdf_search',      # 도구의 이름
    
    # 도구에 대한 설명을 자세히 기입해야 합니다. 
    # LLM에게 알려주는 내용인데.. 이 툴을 언제 쓸지 언제 활용하면 좋을지를 영어로 작성합니다.
    description="use this tool to search information from the PDF document", 
)

### **Agent 가 사용할 도구 목록 정의**

`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다.

In [12]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

### **Agent 생성**

Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [14]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [15]:
prompt = ChatPromptTemplate.from_messages(                  # 프롬프트
    [
        (
            'system',
            'You are a helpful assistant. '
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
        ('placeholder', '{agent_scratchpad}'),
    ]
)

Tool Calling Agent 를 생성합니다.

In [16]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)       # tool calling agent 생성

agent 를 실행하는 AgentExecutor 를 생성합니다.

In [18]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(     
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=False,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

### **에이전트 실행하기**

agent_executor 객체의 invoke() 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. <br>
input 키에 사용자 질문을 값으로 할당한 딕셔너리를 인자로 전달합니다.<br>

In [19]:
answer = agent_executor.invoke({'input': '2024년 프로야구 플레이오프 진출한 5개의 팀을 검색하여 알려주세요'})

In [20]:
answer

{'input': '2024년 프로야구 플레이오프 진출한 5개의 팀을 검색하여 알려주세요',
 'output': '2024년 한국 프로야구 플레이오프에 진출한 팀들은 다음과 같습니다:\n\n1. **KIA 타이거즈** - 정규 시즌 1위\n2. **삼성 라이온즈** - 정규 시즌 2위\n3. **LG 트윈스** - 정규 시즌 3위\n4. **두산 베어스** - 정규 시즌 4위 (와일드카드 결정전 진출)\n5. **kt 위즈** - 정규 시즌 5위 (와일드카드 결정전 진출)\n\n이 팀들은 각각의 성적에 따라 플레이오프에 진출하였으며, KIA 타이거즈는 한국시리즈에 진출한 팀입니다. 플레이오프는 10월 13일부터 시작됩니다.'}

In [21]:
result = agent_executor.stream({'input': '2024년 프로야구 플레이오프 진출한 5개의 팀을 검색하여 알려주세요'})

for step in result:
    print('step:', step)
    print()
    
    if 'actions' in step:
        print('==' * 50)
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

        print('==' * 50)

    elif 'steps' in step:
        print('==' * 50)
        print(f'[steps]')

        for agent_step in step['steps']:
            print("Action:")
            print(f"  Tool: {agent_step.action.tool}")
            print(f"  Tool Input: {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")
        
        print('==' * 50)

    elif 'output' in step:
        print('==' * 50)
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")
                
        print('==' * 50)

step: {'actions': [ToolAgentAction(tool='tavily_search_results_json', tool_input={'query': '2024년 프로야구 플레이오프 진출 팀'}, log="\nInvoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ZhQVhSMBFKVL9l0OH9C8WOJI', 'function': {'arguments': '{"query":"2024년 프로야구 플레이오프 진출 팀"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090'}, id='run-3f3a4b70-679a-4964-878d-305db3ef88f4', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024년 프로야구 플레이오프 진출 팀'}, 'id': 'call_ZhQVhSMBFKVL9l0OH9C8WOJI', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '{"query":"2024년 프로야구 플레이오프 진출 팀"}', 'id': 'call_ZhQVhSMBFKVL9l0OH9C8WOJI', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call

In [22]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0].tool}")
    print(f'===== 도구 호출 =====')

In [23]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [24]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [25]:
result = agent_executor.stream({'input': '2024년 프로야구 플레이오프 진출한 5개의 팀을 검색하여 알려주세요'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)          

===== 도구 호출 =====
사용된 도구 이름 : tavily_search_results_json
===== 도구 호출 =====

===== 관찰 내용 =====
Observation: [{'url': 'https://aldalsuin.tistory.com/entry/2024-%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84-9%EA%B0%9C-%EA%B5%AC%EB%8B%A8%EC%9D%98-%EC%B9%98%EC%97%B4%ED%95%9C-%EC%9A%B0%EC%8A%B9-%EA%B2%BD%EC%9F%81', 'content': '세상만사 리뷰하는 직장인\n\n세상만사 리뷰하는 직장인\n\n티스토리 뷰\n\n2024 프로야구 플레이오프: 9개 구단의 치열한 우승 경쟁\n\n2024 프로야구 플레이오프\n\n2024년 한국 프로야구 시즌이 막바지에 접어들면서, 팬들의 관심은 플레이오프에 집중되고 있습니다. 이번 시즌 플레이오프는 그 어느 때보다 치열한 경쟁이 예상되며, 각 팀들의 전략과 선수들의 활약이 주목받고 있습니다. 이번 글에서는 2024 프로야구 플레이오프의 주요 쟁점과 전망에 대해 자세히 살펴보겠습니다.\n\n\n\n\n\n\n\n2024 프로야구 플레이오프 진출 팀 분석\n\n2024 프로야구 플레이오프에 진출한 팀들의 면면을 살펴보면, 각 팀마다 독특한 강점과 전략이 돋보입니다. 정규시즌 1위 팀부터 와일드카드 진출팀까지, 모든 팀들이 우승을 향한 강한 의지를 보이고 있습니다. [...] 정규시즌 1위 팀은 시즌 내내 안정적인 경기력을 보여주었습니다. 특히 타선의 고른 활약과 선발 투수진의 견고함이 돋보였습니다. 2위 팀은 시즌 후반 무서운 상승세를 타며 1위 팀을 맹추격했습니다. 젊은 선수들의 성장이 팀의 원동력이 되었습니다.\n\n3위 팀은 베테랑 선수들의 경험과 신예 선수들의 패기가 조화를 이루며 좋은 성적을 거두었습니다. 4위 팀은 시즌

In [26]:
from langchain_teddynote.messages import AgentStreamParser

agent_stream_parser = AgentStreamParser()           # 각 단계별 출력을 위한 파서 생성

result = agent_executor.stream(                     # 질의에 대한 답변을 스트리밍으로 출력 요청
    {'input': '2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요.'}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)   # 중간 단계를 parser 를 사용하여 단계별로 출력

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'url': 'https://aldalsuin.tistory.com/entry/2024-%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84-9%EA%B0%9C-%EA%B5%AC%EB%8B%A8%EC%9D%98-%EC%B9%98%EC%97%B4%ED%95%9C-%EC%9A%B0%EC%8A%B9-%EA%B2%BD%EC%9F%81', 'content': '세상만사 리뷰하는 직장인\n\n세상만사 리뷰하는 직장인\n\n티스토리 뷰\n\n2024 프로야구 플레이오프: 9개 구단의 치열한 우승 경쟁\n\n2024 프로야구 플레이오프\n\n2024년 한국 프로야구 시즌이 막바지에 접어들면서, 팬들의 관심은 플레이오프에 집중되고 있습니다. 이번 시즌 플레이오프는 그 어느 때보다 치열한 경쟁이 예상되며, 각 팀들의 전략과 선수들의 활약이 주목받고 있습니다. 이번 글에서는 2024 프로야구 플레이오프의 주요 쟁점과 전망에 대해 자세히 살펴보겠습니다.\n\n\n\n\n\n\n\n2024 프로야구 플레이오프 진출 팀 분석\n\n2024 프로야구 플레이오프에 진출한 팀들의 면면을 살펴보면, 각 팀마다 독특한 강점과 전략이 돋보입니다. 정규시즌 1위 팀부터 와일드카드 진출팀까지, 모든 팀들이 우승을 향한 강한 의지를 보이고 있습니다. [...] 정규시즌 1위 팀은 시즌 내내 안정적인 경기력을 보여주었습니다. 특히 타선의 고른 활약과 선발 투수진의 견고함이 돋보였습니다. 2위 팀은 시즌 후반 무서운 상승세를 타며 1위 팀을 맹추격했습니다. 젊은 선수들의 성장이 팀의 원동력

In [27]:
result = agent_executor.stream({'input': '삼성전자가 자체 개발한 생성형 AI에 대한 정보를 문서에서 검색해서 알려주세요.'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)          

===== 도구 호출 =====
사용된 도구 이름 : pdf_search
===== 도구 호출 =====

===== 관찰 내용 =====
Observation: SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT

In [28]:
from langchain_teddynote.messages import AgentStreamParser

agent_stream_parser = AgentStreamParser()           # 각 단계별 출력을 위한 파서 생성

result = agent_executor.stream(                     # 질의에 대한 답변을 스트리밍으로 출력 요청
    {'input': '삼성전자가 자체 개발한 생성형 AI에 대한 정보를 문서에서 검색해서 알려주세요.'}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)   # 중간 단계를 parser 를 사용하여 단계별로 출력

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 